# Parent Document Retriever

When splitting documents for retrieval, there are often conflicting desires:

- You may want to have small documents, so that their embeddings can most accurately reflect their meaning. If too long, then the embeddings can lose meaning.
- You want to have long enough documents that the context of each chunk is retained.

**Context Enrichment**

The concept here is to retrieve smaller chunks for better search quality, but add up surrounding context for LLM to reason upon.
There are two options — to expand context by sentences around the smaller retrieved chunk or to split documents recursively into a number of larger parent chunks, containing smaller child chunks.

The `ParentDocumentRetriever` strikes that balance by splitting and storing small chunks of data. During retrieval, it first fetches the small chunks but then looks up the parent ids for those chunks and returns those larger documents.

<a href="https://colab.research.google.com/github/edumunozsala/langchain-rag-techniques/blob/main/parent-document-retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install langchain openai tiktoken pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 73.6 MB/s eta 0:00:00


In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.305
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, anyio, async-timeout, dataclasses-json, jsonpatch, langsmith, numexpr, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


### Load the API Key

In [1]:
from dotenv import load_dotenv

# Load the enviroment variables
load_dotenv()

True

### Load the PDF document

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Load a PDF file, extract text into documents and create a FAISS vectorstore with Langchain
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document

import os
import re 

Helper functions to load the PDF file, split it and create the Documents

In [3]:

def split_text_documents(text, source="Not provided", chunk_size=1000, chunk_overlap=0):
    """
    Split the documents in the reader into smaller chuncks.
    
    Args:
    reader (PdfReader): The PdfReader object to be splitted.
    Returns:
    str: The summarized document.
    """
    # Create a text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, separators=[" ", ",", "\n"]
    )
    #Split the text
    texts = text_splitter.split_text(text)
    # Create a list of documents
    docs = [Document(page_content=t, metadata={"source":source, "chunk":i}) for i,t in enumerate(texts)]
    """
    documents = text_splitter.split_documents(documents=reader)
    print(f"Splitted into {len(documents)} chunks")
    # Update the metadata with the source url
    for doc in documents:
        #old_path = doc.metadata["source"]
        #new_url = old_path.replace("langchain-docs", "https:/")
        #doc.metadata.update({"source": new_url})
        print(doc.metadata)
    """
    
    return docs

def load_pdf_from_url(path, files):
    """
    Load one or more PDF documents from the directory in the parameter path.

    Args:
    path: directory where the file or files are located
    files: list of file names

    Returns:
    str: the loaded document
    """
    
    # creating a pdf reader object 
    if path=='' or files=='':
        print('Error: file not found')
        return None
    else:
        reader = PyPDFLoader(os.path.join(path, files[0])) # 'data/Retrieve rerank generate.pdf') 
    
    # printing number of pages in pdf file 
    # print(len(reader.pages)) 
    return reader.load()

def extract_text_from_pdf(path, file):
    """
    Extract and return the text inside a PDF documents in the directory in the parameter path.

    Args:
    path: directory where the file or files are located
    files: list of file names

    Returns:
    str: the the text in the PDF file
    """
    files=[file]
    pages = load_pdf_from_url(path, files)
    
    "Join all pages in one single text"
    text=""
    for page in pages:
        raw_page = re.sub('-\s+', '', page.page_content)
        text += " ".join(raw_page.split())
        text += "\n"
        
    return text

def load_pdf_from_file(path, file, chunk_size, chunk_overlap):
    """
    Load the PDF document file from the directory in the parameter path. Returns a list of Langchain Documents

    Args:
    path: directory where the file or files are located
    file: file name

    Returns:
    lst: list of documents
    """
    
    # creating a pdf reader object 
    if path=='' or file=='':
        print('Error: file not found')
        return None
    else:
        # Read and clean the text in the PDf file
        text= extract_text_from_pdf(path, file)
        # Split the text and create a list of documents
        documents = split_text_documents(text, source=file, chunk_size=chunk_size, chunk_overlap=chunk_overlap)

  
    # printing number of pages in pdf file 
    print(len(documents)) 
    
    return documents


Read the PDf documento of ther user 1

In [5]:
path="data"
file="Attention is all you need.pdf"

docs1= load_pdf_from_file(path, file, 7500, 100)

6


In [6]:
print(len(docs1))

6


## Get the index and Create the Retriever

In [4]:
from langchain.embeddings.openai import OpenAIEmbeddings

from langchain.storage.file_system import LocalFileStore
from langchain.storage._lc_store import create_kv_docstore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.retrievers import ParentDocumentRetriever

We create a function where we create the vector store and build the retriever based on the Parent Document Retriever strategy.

In [5]:
def build_db_retriever():
    # Define the e beddings
    embeddings = OpenAIEmbeddings()
    # Define the parent splitter, parrent size 1,500
    parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)
    # Define the chid splitter
    child_splitter = RecursiveCharacterTextSplitter(chunk_size=300)
    #store = InMemoryStore()
    fs = LocalFileStore("./chroma_db_filestore")
    store = create_kv_docstore(fs)
    # Create the Vectorstore
    vectorstore = Chroma(collection_name="split_parents", embedding_function=embeddings,
                         persist_directory="chroma_db/")
    # Define the Parent Document retriever
    big_chunks_retriever = ParentDocumentRetriever(
        vectorstore=vectorstore,
        docstore=store,
        child_splitter=child_splitter,
        parent_splitter=parent_splitter,
    )
    
    return big_chunks_retriever, vectorstore

Add the docs of user 1 into the index
 

In [12]:
# Build the vectorstore and the retriever
retriever, vectorstore= build_db_retriever()
# Load the documents
# Add the document
retriever.add_documents(docs1)


Let’s now call the vector store search functionality - we should see that it returns small chunks (since we’re storing the small chunks).

In [13]:
sub_docs = vectorstore.similarity_search("convolutional neural network")

In [16]:
print(sub_docs[0].page_content)

Extended Neural GPU [16], ByteNet [ 18] and ConvS2S [ 9], all of which use convolutional neural networks as basic building block, computing hidden representations in parallel for all input and output positions. In these models, the number of operations required to relate signals from two arbitrary


We can also test the retriever, that returns the parent document

In [17]:
# Get the docs
retrieved_docs= retriever.get_relevant_documents("convolutional neural network")

In [18]:
len(retrieved_docs[0].page_content)

1492

In [19]:
print(retrieved_docs[0].page_content)

and instead relying entirely on an attention mechanism to draw global dependencies between input and output. The Transformer allows for signiﬁcantly more parallelization and can reach a new state of the art in translation quality after being trained for as little as twelve hours on eight P100 GPUs. 2 Background The goal of reducing sequential computation also forms the foundation of the Extended Neural GPU [16], ByteNet [ 18] and ConvS2S [ 9], all of which use convolutional neural networks as basic building block, computing hidden representations in parallel for all input and output positions. In these models, the number of operations required to relate signals from two arbitrary input or output positions grows in the distance between positions, linearly for ConvS2S and logarithmically for ByteNet. This makes it more difﬁcult to learn dependencies between distant positions [ 12]. In the Transformer this is reduced to a constant number of operations, albeit at the cost of reduced effect

## Recreate the DB and the Retriever

When your vector store is already created and saved, you can recreate the retriever without needing to split and reload the documents 

In [11]:
# Recreate the vectorstore and the retriever
retriever, vectorstore= build_db_retriever()

Test the retriever

In [12]:
# Get the docs
retrieved_docs= retriever.get_relevant_documents("convolutional neural network")
print(len(retrieved_docs[0].page_content))
print(retrieved_docs[0].page_content)


1492
and instead relying entirely on an attention mechanism to draw global dependencies between input and output. The Transformer allows for signiﬁcantly more parallelization and can reach a new state of the art in translation quality after being trained for as little as twelve hours on eight P100 GPUs. 2 Background The goal of reducing sequential computation also forms the foundation of the Extended Neural GPU [16], ByteNet [ 18] and ConvS2S [ 9], all of which use convolutional neural networks as basic building block, computing hidden representations in parallel for all input and output positions. In these models, the number of operations required to relate signals from two arbitrary input or output positions grows in the distance between positions, linearly for ConvS2S and logarithmically for ByteNet. This makes it more difﬁcult to learn dependencies between distant positions [ 12]. In the Transformer this is reduced to a constant number of operations, albeit at the cost of reduced e

## Build the RAG Chain

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import (
    ConfigurableField,
    RunnableBinding,
    RunnableLambda,
    RunnablePassthrough,
)

Define the Prompt template

In [14]:
# Create the Template 
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
# Create the model for LLM Chain
model = ChatOpenAI(temperature=0)


We can now create the chain using our parent document retriever

In [15]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

Invoke the chain to answer a question from user 1

In [16]:
chain.invoke(
    "How are transformers related to convolutional neural networks?"
)

'Based on the given context, it is mentioned that the Transformer architecture is based solely on attention mechanisms and does not use convolutional neural networks. Therefore, transformers are not directly related to convolutional neural networks.'